In [ ]:
%run ../chap15/NLP_Doc.ipynb
%run ../chap15/BERT_dataset.ipynb
from konlpy.tag import Komoran

import scipy
import time
import re
from openpyxl import Workbook

In [1]:
class transformer(NLP_Doc):
    def __init__(self, name, dataset, hconfigs, JJ=0, show_maps=False, batch_size=1, 
                 l2_decay=0, l1_decay=0, dump_structure=True, word_vector_dimension=512, window_size=1, negative=5, load=0,num_heads=8):
        self.word_vector_dimension=word_vector_dimension
        self.training_unique_words_number=0
        self.training_full_text_number=0
        self.new_words_number=0
        self.new_full_text_number=0
        self.window_size=window_size
        self.negative = negative
        self.num_heads=num_heads
        self.rand_std = 0.03
        self.use_adam=1
        self.inter_stop=0
        self.l2_decay = 0
        self.l1_decay = 0
        self.dic_training=0
        self.is_training=False
        self.text_dataset  = transformer_Dataset(['initial'])
        self.text_dataset.text_find_number_words("text_kor_eng.xlsx",1)
        self.load=load
        self.batch_size=batch_size
        self.look_ahead_index=0
        self.name=name
        self.average_normal=0
        self.ffnn_size=300
        self.warmup_steps=1000
        self.init_parameters(hconfigs)
        self.root_=np.sqrt(int(self.word_vector_dimension/self.num_heads))
        self.n=0
        self.label=np.zeros((self.batch_size,2))
        self.label[:int(self.batch_size//2),0]=1
        self.label[int(self.batch_size//2):,1]=1
        #self.load_parameters('dic')
        #self.G_input_encoder=0
        #self.save_parameters('dic')
        
        
        self.bleu=0

NameError: name 'NLP_Doc' is not defined

In [1]:
def init_parameters(self, hconfigs):
    self.hconfigs = hconfigs
    self.pm_hiddens = []
    pm_hidden=self.alloc_embedding_layer()
    self.pm_hiddens.append(pm_hidden)
    pm_hidden=self.alloc_po_embedding_layer()
    self.pm_hiddens.append(pm_hidden)
    pm_hidden=self.alloc_sep_embedding_layer()
    self.pm_hiddens.append(pm_hidden)
    
    prev_shape=(self.text_dataset.most_length,self.word_vector_dimension)
    for i in range(self.hconfigs[0][1]['repeat']):
        pm_hidden, prev_shape = self.alloc_multi_heads_layer(0)
        self.pm_hiddens.append(pm_hidden)
    
    dic_temp={}
    dic_temp['w'], dic_temp['b']=transformer.making_weight(self,self.word_vector_dimension,len(self.text_dataset.original_id_to_word))
 
    dic_temp['w_cla'], dic_temp['b_cla']=transformer.making_weight(self,self.word_vector_dimension,2)
    self.pm_hiddens.append(dic_temp)
        
transformer.init_parameters = init_parameters

def making_weight_em(self, row, column):
    np.random.seed(int(time.time()*10000000))
    weight= np.random.normal(self.average_normal, self.rand_std, [row, column], dtype = 'float32')
    time.sleep(0.001)
    return weight

def making_weight(self, row, column): #1d
    np.random.seed(int(time.time()*10000000))
    weight= np.random.normal(self.average_normal, self.rand_std, [row, column], dtype = 'float32')
    time.sleep(0.001)
    bias = np.zeros([column], dtype = 'float32')
    return weight, bias

transformer.making_weight_em=making_weight_em
transformer.making_weight=making_weight

NameError: name 'transformer' is not defined

In [ ]:
def alloc_multi_heads_layer(self,decoder=0):
    temp_list=[]
    
    if self.word_vector_dimension%self.num_heads !=0:
        raise NameError('self.word_vector_dimension%self.num_heads')
    
    input_cnt = np_cpu.prod(self.word_vector_dimension)
    output_cnt = int(self.word_vector_dimension/self.num_heads)
    print("self.num_heads:",self.num_heads,output_cnt)
    for i in range(self.num_heads):
        temp_dic={}
        temp_dic[str(i)+'_q'],temp_dic[str(i)+'_qb']=transformer.making_weight(self,input_cnt, output_cnt)
        temp_dic[str(i)+'_k'],temp_dic[str(i)+'_kb']=transformer.making_weight(self,input_cnt, output_cnt)
        temp_dic[str(i)+'_v'],temp_dic[str(i)+'_vb']=transformer.making_weight(self,input_cnt, output_cnt)

        if decoder ==1:
            temp_dic[str(i)+'_q2'],temp_dic[str(i)+'_q2b']=transformer.making_weight(self,input_cnt, output_cnt)
            temp_dic[str(i)+'_k2'],temp_dic[str(i)+'_k2b']=transformer.making_weight(self,input_cnt, output_cnt)
            temp_dic[str(i)+'_v2'],temp_dic[str(i)+'_v2b']=transformer.making_weight(self,input_cnt, output_cnt)
        temp_list.append(temp_dic)
    
    temp_dic={}
    temp_dic['0_w'],temp_dic['0_wb']=transformer.making_weight(self,input_cnt, input_cnt)
    temp_dic['1_w'],temp_dic['1_wb']=transformer.making_weight(self,input_cnt,self.ffnn_size)
    temp_dic['2_w'],temp_dic['2_wb']=transformer.making_weight(self,self.ffnn_size,input_cnt)
    temp_dic['2_batch'],_=transformer.alloc_batch_normal_layer(self,input_cnt,None)
    temp_dic['0_batch'],_=transformer.alloc_batch_normal_layer(self,input_cnt,None)
    if decoder == 1:
        temp_dic['3_w'],temp_dic['3_wb']=transformer.making_weight(self,input_cnt, input_cnt)
        temp_dic['1_batch'],_=transformer.alloc_batch_normal_layer (self,input_cnt,None)
    temp_list.append(temp_dic)
    
    return temp_list, (self.batch_size,self.text_dataset.most_length,self.word_vector_dimension)

transformer.alloc_multi_heads_layer=alloc_multi_heads_layer

In [3]:
def train(self,epoch_count, learning_rate, learning_decrease,restore, name):
    print("the number of words and the number of unique words")
    print(len(self.text_dataset.original_text),self.text_dataset.original_unique_words)
    print(self.text_dataset.original_id_to_word)
    print(self.text_dataset.most_length)
    if self.load == 1:
        self.load_parameters("dic")
        
    
    
    batch_count = int(self.text_dataset.train_count / self.batch_size)
    
    time1 = time2 = int(time.time())
    for epoch in range(epoch_count):
        costs = []
        bleu = []
        bleu_ = []
        self.epoch = epoch+1
        print("self.epoch",self.epoch)
        #self.learning_rate = self.word_vector_dimension**(-0.5)*np.min(np.array([self.epoch**(-0.5),self.epoch*self.warmup_steps**(-1.5)]))
        #print(self.learning_rate)
        self.learning_rate=learning_rate*learning_decrease
        self.text_dataset.shuffle_train_data(self.batch_size*batch_count)
        
        for n in range(batch_count):
            self.n=n
            
            trX = self.text_dataset.get_train_data(self.batch_size, n)
            cost= self.train_step(trX)
            #bleu.append(bleu_item)
            print("loss: ",np.average(cost))
            self.test()
            #if self.epoch%20==0 and self.epoch!=0:
                #bleu_.append(self.bleu_)
            #print("악",trX)
        #print("bleu: ",np.average(np.array(bleu)))
        #if self.epoch%20==0 and  self.epoch!=0:
            #print("bleu_real: ",np.average(np.array(bleu_)))
        if epoch % 5 ==0and epoch !=0:
            save_parameters(self,'dic')
        
    tm_total = int(time.time()) - time1
    print('Model {} train ended in {} secs:'.format(self.name, tm_total))

transformer.train = train

NameError: name 'transformer' is not defined

In [ ]:
def save_parameters(self,char):    #for word vector

    time1 = int(time.time())

    with open("hyper_"+self.name+".csv",'w',newline='') as aa:
        
        print(self.hconfigs, file=aa)
        print(self.word_vector_dimension, file=aa)
        print(self.training_unique_words_number,file=aa)
        print(self.training_full_text_number,file=aa)
        print(self.new_words_number,file=aa)
        print(self.new_full_text_number,file=aa)
        print(self.batch_size,file=aa)
        print(self.ffnn_size,file=aa)
        print(self.pm_hiddens[0]['dic'].shape,file=aa)
        
        #print(self.pm_hiddens)
    with open("dic_original_"+self.name+"_.csv",'w',newline='') as aa:
        for i in range(len(self.text_dataset.original_id_to_word)):
            print(self.text_dataset.original_id_to_word[i], file=aa)
            
    self.save_ass("word_vector_original_",self.pm_hiddens[0]['dic'])
    
    self.save_ass("word_vector_pos_",self.pm_hiddens[1]['dic'])
    
    self.save_ass("word_vector_sep_",self.pm_hiddens[2]['dic'])
    
    
    
    for j in range(self.hconfigs[0][1]['repeat']):
        pm=self.pm_hiddens[j+3]
        for i in range(self.num_heads):
            self.save_ass("encoder"+str(j)+str(i)+"below_q",pm[i][str(i)+'_q'])
            self.save_ass("encoder"+str(j)+str(i)+"below_k",pm[i][str(i)+'_k'])
            self.save_ass("encoder"+str(j)+str(i)+"below_v",pm[i][str(i)+'_v'])
            self.save_ass_1("encoder"+str(j)+str(i)+"below_qb",pm[i][str(i)+'_qb'])
            self.save_ass_1("encoder"+str(j)+str(i)+"below_kb",pm[i][str(i)+'_kb'])
            self.save_ass_1("encoder"+str(j)+str(i)+"below_vb",pm[i][str(i)+'_vb'])

        self.save_ass("encoder"+str(j)+"upper_0_w",pm[-1]['0_w'])
        self.save_ass("encoder"+str(j)+"upper_1_w",pm[-1]['1_w'])
        self.save_ass("encoder"+str(j)+"upper_2_w",pm[-1]['2_w'])
        self.save_ass_1("encoder"+str(j)+"upper_0_wb",pm[-1]['0_wb'])
        self.save_ass_1("encoder"+str(j)+"upper_1_wb",pm[-1]['1_wb'])
        self.save_ass_1("encoder"+str(j)+"upper_2_wb",pm[-1]['2_wb'])
        self.save_ass_1("encoder"+str(j)+"upper_0_batch_mavg",pm[-1]['0_batch']['mavg'])
        self.save_ass_1("encoder"+str(j)+"upper_0_batch_mvar",pm[-1]['0_batch']['mvar'])
        self.save_ass_1("encoder"+str(j)+"upper_2_batch_mavg",pm[-1]['2_batch']['mavg'])
        self.save_ass_1("encoder"+str(j)+"upper_2_batch_mvar",pm[-1]['2_batch']['mvar'])

    
 
            
  
    self.save_ass("output_",self.pm_hiddens[-1]['w'])
    self.save_ass_1("output_b",self.pm_hiddens[-1]['b'])
   
    
            
            
def save_ass(self,name,pm):
    with open(name+"_.csv",'w',newline='') as aa:
        temp_list=list()
        for i in range(pm.shape[0]):
            temp_list=list()
            re_before=pm[i,:].tolist()
            for j in range(len(re_before)):
                re_before[j]=re.sub('\\n','',str(re_before[j]))
                temp_list.append(re_before[j])
            print(temp_list, file=aa)

def save_ass_1(self,name,pm):
    with open(name+"_.csv",'w',newline='') as aa:
        re_before=pm.tolist()
        for j in range(len(re_before)):
            re_before[j]=re.sub('\\n','',str(re_before[j]))
 
        print(re_before, file=aa)
            
def load_parameters(self,char): #for word vector
    self.load_ass("word_vector_original_",self.pm_hiddens[0]['dic'])
    self.load_ass("word_vector_po_",self.pm_hiddens[1]['dic'])
    self.load_ass("word_vector_sep_",self.pm_hiddens[2]['dic'])
    
    for j in range(self.hconfigs[0][1]['repeat']):
        pm=self.pm_hiddens[j+1]
        for i in range(self.num_heads):
            self.load_ass("encoder"+str(j)+str(i)+"below_q",pm[i][str(i)+'_q'])
            self.load_ass("encoder"+str(j)+str(i)+"below_k",pm[i][str(i)+'_k'])
            self.load_ass("encoder"+str(j)+str(i)+"below_v",pm[i][str(i)+'_v'])
            self.load_ass_1("encoder"+str(j)+str(i)+"below_qb",pm[i][str(i)+'_qb'])
            self.load_ass_1("encoder"+str(j)+str(i)+"below_kb",pm[i][str(i)+'_kb'])
            self.load_ass_1("encoder"+str(j)+str(i)+"below_vb",pm[i][str(i)+'_vb'])

        self.load_ass("encoder"+str(j)+"upper_0_w",pm[-1]['0_w'])
        self.load_ass("encoder"+str(j)+"upper_1_w",pm[-1]['1_w'])
        self.load_ass("encoder"+str(j)+"upper_2_w",pm[-1]['2_w'])
        self.load_ass_1("encoder"+str(j)+"upper_0_wb",pm[-1]['0_wb'])
        self.load_ass_1("encoder"+str(j)+"upper_1_wb",pm[-1]['1_wb'])
        self.load_ass_1("encoder"+str(j)+"upper_2_wb",pm[-1]['2_wb'])
        self.load_ass_1("encoder"+str(j)+"upper_0_batch_mavg",pm[-1]['0_batch']['mavg'])
        self.load_ass_1("encoder"+str(j)+"upper_0_batch_mvar",pm[-1]['0_batch']['mvar'])
        self.load_ass_1("encoder"+str(j)+"upper_2_batch_mavg",pm[-1]['2_batch']['mavg'])
        self.load_ass_1("encoder"+str(j)+"upper_2_batch_mvar",pm[-1]['2_batch']['mvar'])
    
 
            
    self.load_ass("output_",self.pm_hiddens[-1]['w'])
    self.load_ass_1("output_b",self.pm_hiddens[-1]['b'])


def load_ass(self,name,pm):
    print(name)
    with open(name+"_.csv",'r',newline='') as aa:      
        for i in range(pm.shape[0]):
            temp_line=aa.readline().split(',')
        
            for j in range(len(temp_line)):
                if j ==len(temp_line)-1:
                    try:
                        pm[i,j]=float(temp_line[j][2:-3])
                    except:
                        print(temp_line)
                else:
                    try:
                        pm[i,j]=float(temp_line[j][2:-1])
                    except:
                        print(temp_line)

def load_ass_1(self,name,pm):
    print(name)
    with open(name+"_.csv",'r',newline='') as aa:      
        temp_line=aa.readline().split(',')

        for j in range(len(temp_line)):
            if j ==len(temp_line)-1:
                pm[j]=float(temp_line[j][2:-3])
            else:
                pm[j]=float(temp_line[j][2:-1])
    
transformer.load_ass_1=load_ass_1
transformer.load_ass=load_ass
transformer.save_ass_1=save_ass_1
transformer.save_ass=save_ass
transformer.load_parameters=load_parameters
transformer.save_parameters=save_parameters

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
def train_step(self, x):
    loss_temp=list()
    y=0
    self.first_time=1
    self.is_training = True
    loss, aux_nn = self.forward_neuralnet(x,y)
    loss_temp.append(loss)
    G_loss = 1.0
    self.backprop_neuralnet(G_loss, aux_nn)
    self.is_training = False
    
    
    return np.mean(np.array(loss_temp))

transformer.train_step=train_step

In [ ]:
def forward_neuralnet(self, x,y):
    hidden = x
    #print(x[0])
    aux_layers = [] #embedding, encoder, decoder 1ch/  embedding [interlalyer] output
    hidden_1, aux = self.forward_embedding_layer(hidden, None, self.pm_hiddens[0])
    
    aux_layers.append(aux)
    hidden_2, aux = self.forward_po_embedding_layer(hidden, None, self.pm_hiddens[1])
    aux_layers.append(aux)
    hidden_3, aux = self.forward_sep_embedding_layer(hidden, None, self.pm_hiddens[2])
    #print(self.pm_hiddens[0])
    aux_layers.append(aux)
    #print(hidden_1.shape,hidden_2.shape,hidden_3.shape)
    hidden =hidden_1+hidden_2+hidden_3
    hidden=self.forward_mask_layer(hidden)
    #for i in range(hidden.shape[0]):
    ##    for j in range(hidden.shape[1]):
    #        print(hidden[i,j])
    aux_layers_temp=[]
    
    for i in range(self.hconfigs[0][1]['repeat']):
        hidden, aux = self.forward_multi_heads_layer(hidden, self.pm_hiddens[i+3],0)
        aux_layers_temp.append(aux)

    aux_layers.append(aux_layers_temp)
    aux_layers.append(hidden)
    output=np.matmul(hidden[:,self.number,:],self.pm_hiddens[-1]['w'])+self.pm_hiddens[-1]['b']
    self.output_cla=np.matmul(hidden[:,0,:],self.pm_hiddens[-1]['w_cla'])+self.pm_hiddens[-1]['b_cla']
    #print(self.output_cla.shape)
    #print(output)
    
    self.final_cla=softmax_cross_entropy_with_logits(self.label , self.output_cla)
    #print(self.G_input_cla.shape)
    y=x[:,self.number]
          
    result__=np.zeros(np.array(y).shape)
    label=self.text_dataset.training_original_one_hot_word[y]
    #print(label.shape)
    for i in range(y.shape[0]):
        result__[i]=softmax_cross_entropy_with_logits(label[i]
                                                              , output[i,:,:])
    #print(result__[i])
    return np.mean(result__), [aux_layers, output, result__,self.text_dataset.training_original_one_hot_word[y]]

transformer.forward_neuralnet=forward_neuralnet

In [1]:
def backprop_neuralnet(self, G_loss, aux):
    #print(G_loss)
    aux_layers, output, result__, y= aux
    
    g_loss_entropy = 1.0 / np_cpu.prod(result__.shape)
    g_entropy_output=np.zeros(y.shape)
    for i in range(y.shape[0]):
        
        g_entropy_output[i] = softmax_cross_entropy_with_logits_derv(y[i,:,:], output[i,:,:])
    G_entropy = g_loss_entropy * G_loss
    G_affine = g_entropy_output * G_entropy
    G_bias=np.sum(G_affine, axis = (0,1))
    g_affine_input = self.pm_hiddens[-1]['w'].transpose()
    G_weight_=np.zeros_like(self.pm_hiddens[-1]['w'])
    G_hidden=np.zeros_like(aux_layers[-1])
    
    #------------------------
    
    #------------------------
    #print(G_hidden)
    #print(G_affine.shape)
    #print(G_weight_.shape)
    #print(aux_layers[-1].shape)
    #print(aux_layers[-1][0,self.number,:].transpose().shape)
    #print(np.matmul(aux_layers[-1][i,self.number,:].transpose(),G_affine[i]).shape)
    for i in range(G_affine.shape[0]):
        if i ==0:
            G_weight_=np.matmul(aux_layers[-1][i,self.number,:].transpose(),G_affine[i])
        else:
            G_weight_+=np.matmul(aux_layers[-1][i,self.number,:].transpose(),G_affine[i])

    self.update_param(self.pm_hiddens[-1], 'w', G_weight_)
    self.update_param(self.pm_hiddens[-1], 'b', G_bias)
    G_hidden[:,self.number,:] = np.matmul(G_affine, g_affine_input)
    #print(G_hidden)
    #---------------------
    g_loss_entropy = 1.0 / np_cpu.prod(self.final_cla.shape)
    g_entropy_output=np.zeros(self.label.shape)
    #print(self.label.shape, self.output_cla.shape)
    
    g_entropy_output = softmax_cross_entropy_with_logits_derv(self.label, self.output_cla)
    G_entropy = g_loss_entropy * G_loss
    G_affine = g_entropy_output * G_entropy
    #print(aux_layers[-1][:,0,:].transpose().shape,G_affine.shape)
    G_bias=np.sum(G_affine, axis = 0)
    g_affine_input = self.pm_hiddens[-1]['w_cla'].transpose()
    G_weight_=np.zeros_like(self.pm_hiddens[-1]['w_cla'])
    
    G_weight_=np.matmul(aux_layers[-1][:,0,:].transpose(),G_affine)
        

    self.update_param(self.pm_hiddens[-1], 'w_cla', G_weight_)
    self.update_param(self.pm_hiddens[-1], 'b_cla', G_bias)
    G_hidden[:,0,:] = np.matmul(G_affine, g_affine_input)
    #---------------
    aux_layers_temp=aux_layers[-2]
    
    for i in reversed(range(self.hconfigs[0][1]['repeat'])):
        pm, aux = self.pm_hiddens[i+3], aux_layers_temp[i]
        G_hidden = self.backprop_multi_heads_layer(G_hidden, pm,aux,0)
    #print(G_hidden)
    G_hidden=self.backprop_mask_layer(G_hidden)
    #print(G_hidden)
    aux_layers_temp=aux_layers[2]
    _ = self.backprop_sep_embedding_layer(G_hidden, None, self.pm_hiddens[2],aux_layers_temp)
    aux_layers_temp=aux_layers[1]
    _ = self.backprop_po_embedding_layer(G_hidden, None, self.pm_hiddens[1],aux_layers_temp)
    aux_layers_temp=aux_layers[0]
    _ = self.backprop_embedding_layer(G_hidden, None, self.pm_hiddens[0],aux_layers_temp)

    return G_hidden

transformer.backprop_neuralnet=backprop_neuralnet

NameError: name 'transformer' is not defined

In [1]:
def forward_multi_heads_layer(self, x, pm,decoder=0):
    
    aux_layers=[]
    for i in range(self.num_heads):
        q_matrix=np.matmul(x,pm[i][str(i)+'_q'])+pm[i][str(i)+'_qb']
        k_matrix=np.matmul(x,pm[i][str(i)+'_k'])+pm[i][str(i)+'_kb']
        v_matrix=np.matmul(x,pm[i][str(i)+'_v'])+pm[i][str(i)+'_vb']
        #print("!",q_matrix.shape, x.shape)
        list_before_conca=list()
        after_softmax_temp=[]
        for j in range(self.batch_size):
            before_softmax=np.matmul(q_matrix[j,:,:],k_matrix[j,:,:].transpose())\
            /np.sqrt(int(self.word_vector_dimension/self.num_heads))
            #print(q_matrix[j,:,:].shape,k_matrix[j,:,:].shape)
            if decoder ==1:
                
                pass
                    
            else:
                
                for k in range(len(self.input_mask[0])-self.text_dataset.most_length):
                    if self.input_mask[j,k]==1:
                        before_softmax[:,k:self.text_dataset.most_length]=(-1e9)
        
                        break
                for k in range(len(self.input_mask[0])-self.text_dataset.most_length):
                    if self.input_mask[j,k+self.text_dataset.most_length]==1:
                        before_softmax[:,k+self.text_dataset.most_length:]=(-1e9)
        
                        break

            #print(before_softmax)
            after_softmax = softmax(before_softmax)
            temp_result=np.matmul(after_softmax,v_matrix[j,:,:])
            list_before_conca.append(temp_result)
            after_softmax_temp.append(after_softmax)

        aux_layers.append([x, q_matrix,k_matrix,before_softmax,\
                                      np.array(after_softmax_temp),v_matrix])
        list_before_conca=np.array(list_before_conca)
        
        if i == 0:
            final_attention_conca=list_before_conca
        else:
            final_attention_conca = np.concatenate((final_attention_conca,list_before_conca),axis=2)
    

    aux_layers.append(final_attention_conca)#1

    final_attention_conca=np.matmul(final_attention_conca,pm[-1]['0_w'])+pm[-1]['0_wb']
  
    final_attention_conca_temp,temp=self.forward_batch_normal_layer(final_attention_conca+x,None,pm[-1]['0_batch']) #+x
    aux_layers.append(temp) #6#3
    aux_layers.append(final_attention_conca) #2
    #atteintion_last
    
    
    
    final_attention_conca=relu(np.matmul(final_attention_conca_temp\
                    ,pm[-1]['1_w'])+pm[-1]['1_wb'])
    aux_layers.append(final_attention_conca) #-3
    #print(final_attention_conca_temp.shape,pm[-1]['1_w'].shape)
    final_attention_conca=np.matmul(final_attention_conca\
                    ,pm[-1]['2_w'])+pm[-1]['2_wb']
    aux_layers.append(final_attention_conca) #-2
    final_attention_conca,temp = self.forward_batch_normal_layer(final_attention_conca+final_attention_conca_temp,None,pm[-1]['2_batch']) #+final_attention_conca_temp
    aux_layers.append(temp) #-1

    return final_attention_conca,aux_layers

transformer.forward_multi_heads_layer=forward_multi_heads_layer

NameError: name 'transformer' is not defined

In [2]:
def backprop_multi_heads_layer(self, G_input, pm, aux,decoder):

    if pm is None: return G_y
    
    
    G_input_shape=G_input.shape
    G_input=G_input.reshape(-1)
    G_input=np.where(G_input<1e2,G_input,1e2)
    G_input=np.where(G_input<-1e2,-1e2,G_input)
    G_input=np.where(np.abs(G_input)<1e-5,0,G_input)
    G_input=G_input.reshape(G_input_shape)
    
    G_input = self.backprop_batch_normal_layer(G_input,None,pm[-1]['2_batch'],aux[-1])
    #print(G_input)
    new_G_input=np.copy(G_input)                          
    g_affine_input = pm[-1]['2_w'].transpose()
    
    for i in range(G_input.shape[0]):
        if i ==0:
            G_weight_=np.matmul(aux[-3][i].transpose(),G_input[i])
        else:
            G_weight_+=np.matmul(aux[-3][i].transpose(),G_input[i])
    #print(g_affine_input.shape,G_input.shape,aux[-3][0].transpose().shape,G_weight_.shape)
    G_bias=np.sum(G_input, axis = (0,1))
    self.update_param(pm[-1], '2_wb', G_bias)
    self.update_param(pm[-1], '2_w', G_weight_)
    
    G_input = np.matmul(G_input, g_affine_input)
    G_input = G_input*relu_derv(G_input)

    g_affine_input = pm[-1]['1_w'].transpose()
    #print(G_input.shape, aux[-4].shape)
    for i in range(G_input.shape[0]):
        if i ==0:
            G_weight_=np.matmul(aux[-4][i].transpose(),G_input[i])
        else:
            G_weight_+=np.matmul(aux[-4][i].transpose(),G_input[i])
    G_bias=np.sum(G_input, axis = (0,1))
    self.update_param(pm[-1], '1_wb', G_bias)
    self.update_param(pm[-1], '1_w', G_weight_)
    G_input_second = np.matmul(G_input, g_affine_input)+new_G_input

    #print(G_input_second)
    
    #print(G_input_second)
    G_input_second_shape=G_input_second.shape
    G_input_second=G_input_second.reshape(-1)
    
    G_input_second=np.where(G_input_second<1e2,G_input_second,1e2)
    G_input_second=np.where(G_input_second<-1e2,-1e2,G_input_second)
    G_input_second=np.where(np.abs(G_input_second)<1e-5,0,G_input_second)
    
    G_input_second=G_input_second.reshape(G_input_second_shape)
    G_input_second = self.backprop_batch_normal_layer(G_input_second,None,pm[-1]['0_batch'],aux[self.num_heads+1])
    
    new_G_input=np.copy(G_input_second)
    
    
    g_affine_input = pm[-1]['0_w'].transpose()

    for i in range( G_input_second.shape[0]):
        if i ==0:
            G_weight_=np.matmul(aux[self.num_heads][i].transpose(),G_input_second[i])
        else:
            G_weight_+=np.matmul(aux[self.num_heads][i].transpose(),G_input_second[i])
            
    G_bias=np.sum(G_input_second, axis = (0,1))
    self.update_param(pm[-1], '0_wb', G_bias)
    self.update_param(pm[-1], '0_w', G_weight_)
    #print(G_bias)
    
    G_input_final=np.zeros_like(G_input_second)
    G_input_ = np.matmul(G_input_second, g_affine_input)
    #print(G_input_)
    for i in range(self.num_heads):

        G_input_temp=G_input_[:,:,(G_input_.shape[-1]/self.num_heads)*i:(G_input_.shape[-1]/self.num_heads)*(i+1)]
        aux_temp_num=aux[i]
        for j in range(self.batch_size):
            if j == 0:
                g_affine_weight = aux_temp_num[-2][j,:,:].transpose()
                g_affine_input = aux_temp_num[-1][j,:,:].transpose()
                G_v = np.matmul(g_affine_weight, G_input_temp[j,:,:])
                G_input = np.matmul(G_input_temp[j,:,:], g_affine_input)
                G_input = G_input*softmax_derv(G_input)
                #print(G_input)
                #print(G_input)
                if decoder ==1:

                    pass
                    
                else:
                    for k in range(len(self.input_mask[0])-self.text_dataset.most_length):
                        if self.input_mask[j,k]==1:
                            G_input[:,k:self.text_dataset.most_length]=0

                            break
                    for k in range(len(self.input_mask[0])-self.text_dataset.most_length):
                        if self.input_mask[j,k+self.text_dataset.most_length]==1:
                            G_input[:,k+self.text_dataset.most_length:]=0

                            break
                    
                    
                    
                G_input=G_input/self.root_
                g_affine_weight = aux_temp_num[1][j,:,:].transpose()
                g_affine_input = aux_temp_num[2][j,:,:]
                G_k = np.matmul(g_affine_weight, G_input).transpose()
                G_q = np.matmul(G_input, g_affine_input)
         
                G_v = G_v[np.newaxis,:,:]
                G_k = G_k[np.newaxis,:,:]
                G_q = G_q[np.newaxis,:,:]
    
            else:
                g_affine_weight = aux_temp_num[-2][j,:,:].transpose()
                g_affine_input = aux_temp_num[-1][j,:,:].transpose()
                G_v_temp=np.matmul(g_affine_weight, G_input_temp[j,:,:])
                G_v_temp=G_v_temp[np.newaxis,:,:]
                G_input = np.matmul(G_input_temp[j,:,:], g_affine_input)
                G_input = G_input*softmax_derv(G_input)
                if decoder ==1:

                    for k in range(len(self.input_mask_target[0])):
                        G_input[k,k+1:]=0
                    #print(G_input)
                    
                    for k in range(len(self.input_mask_target[0])):
                        if self.input_mask_target[j,k]==0:
                            G_input[:,k:]=0
                            break
                    
                else:
                    
                    
                    for k in range(len(self.input_mask[0])):
                        if self.input_mask[j,k]==1:
                            G_input[:,k:]=0
                            break
                    
                #print(G_input)
                G_input=G_input/self.root_
                
                g_affine_weight = aux_temp_num[1][j,:,:].transpose()
                g_affine_input = aux_temp_num[2][j,:,:]
                G_k_temp = np.matmul(g_affine_weight, G_input).transpose()
                G_k_temp=G_k_temp[np.newaxis,:,:]
                G_q_temp = np.matmul(G_input, g_affine_input)
                G_q_temp=G_q_temp[np.newaxis,:,:]
                G_q = np.concatenate((G_q, G_q_temp),axis=0)
                G_v = np.concatenate((G_v, G_v_temp),axis=0)
                G_k = np.concatenate((G_k, G_k_temp),axis=0)
     
    
        
        for j in range(self.batch_size):
            g_affine_weight = aux_temp_num[0][j].transpose()
   
            if j==0:
                G_weight_8 = np.matmul(g_affine_weight, G_k[j,:,:])
            else:
                G_weight_8 += np.matmul(g_affine_weight, G_k[j,:,:])

            if j==0:
                G_weight_9 = np.matmul(g_affine_weight, G_v[j,:,:])
            else:
                G_weight_9 += np.matmul(g_affine_weight, G_v[j,:,:])
 
            if j==0:
                G_weight_10= np.matmul(g_affine_weight, G_q[j,:,:])
            else:
                G_weight_10+= np.matmul(g_affine_weight, G_q[j,:,:])
   
        #print(G_q)
    
        
        G_bias=np.sum(G_q, axis = (0,1))
        #print(G_bias)
        self.update_param(pm[i], str(i)+'_qb', G_bias)
        
        G_bias=np.sum(G_k, axis = (0,1))
        self.update_param(pm[i], str(i)+'_kb', G_bias)
        G_bias=np.sum(G_v, axis = (0,1))
        self.update_param(pm[i], str(i)+'_vb', G_bias)
        
        

        g_affine_input = pm[i][str(i)+'_k'].transpose()
        G_input_final += np.matmul(G_k, g_affine_input)
        g_affine_input = pm[i][str(i)+'_v'].transpose()
        G_input_final += np.matmul(G_v, g_affine_input)
        g_affine_input = pm[i][str(i)+'_q'].transpose()
        G_input_final += np.matmul(G_q, g_affine_input)
        self.update_param(pm[i], str(i)+'_k', G_weight_8)
        self.update_param(pm[i], str(i)+'_q', G_weight_10)
        self.update_param(pm[i], str(i)+'_v', G_weight_9)
    G_input_final += new_G_input
    
    return G_input_final
    

transformer.backprop_multi_heads_layer=backprop_multi_heads_layer

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-2-ce694683a438>, line 301)

In [ ]:
def test(self,a='is', b='are'):
    a_=self.text_dataset.original_word_to_id[a]
    b_=self.text_dataset.original_word_to_id[b]

    print(np.dot(self.pm_hiddens[0]['dic'][a_], self.pm_hiddens[0]['dic'][b_])/((sum(self.pm_hiddens[0]['dic'][a_]*self.pm_hiddens[0]['dic'][a_])\
                                                                                **0.5)*(sum(self.pm_hiddens[0]['dic'][b_]*self.pm_hiddens[0]['dic'][b_])**0.5)))

def predict_translation(self,sentence_1,t):

    result_list=list()
    temp_list=list()
    sentence_2=self.predict_forward_neuralnet(sentence_1,t)

    for i in range(self.batch_size):
        temp_list_list=list()
        result_list_list=list()
        for w in sentence_1.tolist()[i]:
            if self.text_dataset.original_unique_words==w:
                pass
            else:
                temp_list_list.append(self.text_dataset.original_id_to_word[w])
        temp_list.append(temp_list_list)
        
        for w in sentence_2.tolist()[i]:
            result_list_list.append(self.text_dataset.target_id_to_word[w])
        result_list.append(result_list_list)

        print(temp_list[i],"->",result_list[i])
        print(" ")


transformer.test=test
transformer.predict_translation=predict_translation

In [2]:
def alloc_embedding_layer(self):
    input_cnt = len(self.text_dataset.original_id_to_word)
    output_cnt = self.word_vector_dimension
    a = transformer.making_weight_em(self,input_cnt, output_cnt)
    self.G_original_embedding_like=np.zeros_like(a)
    return {'dic':a}

def forward_embedding_layer(self, x, hconfig, pm):
    output=pm['dic'][x,:]
    self.input_mask=np.sign(x-len(self.text_dataset.original_id_to_word)+2)+1
    self.input_mask[:,0]=0
    return output, x

def backprop_embedding_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y
    x= aux
    #print(G_y)
    for i in range(self.batch_size):
        self.G_original_embedding_like[x[i],:]+=G_y[i]
    self.update_param(pm, 'dic', self.G_original_embedding_like)
    self.G_original_embedding_like=np.zeros_like(self.G_original_embedding_like)
    
    return 0
transformer.alloc_embedding_layer=alloc_embedding_layer
transformer.forward_embedding_layer =forward_embedding_layer
transformer.backprop_embedding_layer=backprop_embedding_layer

def alloc_po_embedding_layer(self):
    input_cnt = self.text_dataset.most_length*2
    output_cnt = self.word_vector_dimension
    a = transformer.making_weight_em(self,input_cnt, output_cnt)
    self.G_po_embedding_like=np.zeros_like(a)
    return {'dic':a}

def forward_po_embedding_layer(self, x, hconfig, pm):
    output=np.tile(pm['dic'],self.batch_size).reshape((self.batch_size,\
                                                                            pm['dic'].shape[0],pm['dic'].shape[1]))
    #print(output.shape)
    return output, x

def backprop_po_embedding_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y
    x= aux
    #print(self.G_po_embedding_like.shape,G_.y.shape)
    #self.G_po_embedding_like=np.sum(G_y
    for i in range(self.batch_size):
        self.G_po_embedding_like+=G_y[i]
    self.update_param(pm, 'dic', self.G_po_embedding_like)
    self.G_po_embedding_like=np.zeros_like(self.G_po_embedding_like)
    return 0
transformer.alloc_po_embedding_layer=alloc_po_embedding_layer
transformer.forward_po_embedding_layer =forward_po_embedding_layer
transformer.backprop_po_embedding_layer=backprop_po_embedding_layer

def alloc_sep_embedding_layer(self):
    input_cnt = 2
    output_cnt = self.word_vector_dimension
    a = transformer.making_weight_em(self,input_cnt, output_cnt)
    self.G_sep_embedding_like=np.zeros_like(a)
    return {'dic':a}

def forward_sep_embedding_layer(self, x, hconfig, pm):
    
    output_1=np.tile(pm['dic'][0,:],2*self.text_dataset.most_length*int(self.batch_size//2)).reshape((self.batch_size//2,\
                                                                            2*self.text_dataset.most_length,pm['dic'][0,:].shape[0]))
    
    output_2=np.tile(pm['dic'][1,:],2*self.text_dataset.most_length*(self.batch_size-int(self.batch_size//2))).reshape((self.batch_size-int(self.batch_size//2),2*self.text_dataset.most_length,pm['dic'][1,:].shape[0]))
    output=np.concatenate((output_1,output_2),axis=0)
    #print("ewr",output.shape)
    return output, x

def backprop_sep_embedding_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y
    x= aux
    sum_=0
    sum_1=0
    #print(self.G_sep_embedding_like[0,:].shape)
    
    for i in range(self.batch_size):
        sum_+=G_y[i,:self.text_dataset.most_length,:]
        sum_1+=G_y[i,self.text_dataset.most_length:,:]
    #print(sum_.shape)
    self.G_sep_embedding_like[0,:]+=np.sum(sum_,axis=0)
    self.G_sep_embedding_like[1,:]+=np.sum(sum_1,axis=0)
    
    self.update_param(pm, 'dic', self.G_sep_embedding_like)
    self.G_sep_embedding_like=np.zeros_like(self.G_sep_embedding_like)
 
    return 0
transformer.alloc_sep_embedding_layer=alloc_sep_embedding_layer
transformer.forward_sep_embedding_layer =forward_sep_embedding_layer
transformer.backprop_sep_embedding_layer=backprop_sep_embedding_layer

NameError: name 'transformer' is not defined

In [ ]:


def forward_mask_layer(self, x):
    self.number=np.random.randint(1,2*self.text_dataset.most_length,size=int(2*self.text_dataset.most_length*0.15))
    x[:,self.number,:]=0

    
    return x

def backprop_mask_layer(self, x):
    return x

transformer.forward_mask_layer =forward_mask_layer
transformer.backprop_mask_layer=backprop_mask_layer